1. predict submission1 with word2vec
2. predict submission2 with DeepFM 
3. combine recommender 20 items in the submission1 and submission2


# 1.load lib and data

due to memory problem, i'll use parquet file by @RADEK OSMULSKI 

In [ ]:
!pip install polars
!pip install pickle5

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json
import polars as pl

from datetime import timedelta
from tqdm.notebook import tqdm
from collections import Counter
from heapq import nlargest


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# #paths

# data_path = Path("/kaggle/input/otto-recommender-system")
# train_df = data_path/'train.jsonl'
# test_df = data_path/'test.jsonl'
# sample_sub_path = Path('/kaggle/input/otto-recommender-system/sample_submission.csv')


In [ ]:
train = pd.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pd.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')



In [ ]:
import pickle5 as pickle

with open('../input/otto-full-optimized-memory-footprint/id2type.pkl', "rb") as fh:
    id2type = pickle.load(fh)
with open('../input/otto-full-optimized-memory-footprint/type2id.pkl', "rb") as fh:
    type2id = pickle.load(fh)

In [ ]:
train_type_as_string = train.iloc[:10000000].type.map(lambda i: id2type[i])
type_as_string.head()

test_type_as_string = test.iloc[:5015409].type.map(lambda i : id2type[i])


In [ ]:
train['type_id'] = train_type_as_string
test['type_id'] = test_type_as_string

In [ ]:
train = train[:len(train_type_as_string)]
test = test[:len(test_type_as_string)]

In [ ]:
train['type_id'].value_counts()

#let's make test data

In [ ]:
train_sample = train[:1000000]
test_sample = test[:1000000]

## 1-1.json to DF

reference of read data :

https://www.kaggle.com/code/columbia2131/otto-read-a-chunk-of-jsonl-to-manageable-df?scriptVersionId=109793998 & https://www.kaggle.com/code/takaito/otto-word2vec-tutorial

- due to memory problem, i will use parquet file for data analyze
- how much faster parquet than pandas? 
- check here https://dadosdadosdados.wordpress.com/2015/12/30/benchmarking-csv-vs-parquet/ 

In [ ]:
# def read_jsonl(target: str) -> pd.DataFrame():
#     sessions = pd.DataFrame()
#     chunks = pd.read_json(data_path / f'{target}.jsonl', lines=True, chunksize=1000000)

#     for e, chunk in enumerate(chunks):
#         event_dict = {
#             'session': [],
#             'aid': [],
#             'ts': [],
#             'type': [],
#         }
#         if e < 2:
#             for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
#                 for event in events:
#                     event_dict['session'].append(session)
#                     event_dict['aid'].append(event['aid'])
#                     event_dict['ts'].append(event['ts'])
#                     event_dict['type'].append(event['type'])
#             chunk_session = pd.DataFrame(event_dict)
#             sessions = pd.concat([sessions, chunk_session])
#         else:
#             break
#     return sessions.reset_index(drop=True)
# train_sessions = read_jsonl('train') 
# test_sessions = read_jsonl('test') 

In [ ]:
# type2id = {'clicks' :0, "carts" : 1, 'orders' : 2}

# train_sessions['type2id'] = train_sessions['type'].apply(lambda x : type2id[x])
# test_sessions['type2id'] = test_sessions['type'].apply(lambda x : type2id[x])

## 1-2.calculating the scores used for co-visitation matrix
- this variable could be use for lgbm or xgboost

reference : https://www.kaggle.com/code/radek1/co-visitation-matrix-simplified-imprvd-logic, https://www.kaggle.com/code/alberteinsten/cudf-pandas-proof-of-concept-lgbm-ranker

In [ ]:
# def add_session_length(df) :
#     df['session_length'] = train_sessions.groupby('session')['ts'].transform('count')
#     return df

# def add_action_num_reverse_chrono(df):
#     df['action_num_reverse_chrono'] = df.session_length - df.groupby('session').cumcount() - 1
#     return df

# def add_log_recency_score(df):
#     linear_interpolation = 0.1 + ((1-0.1) / (df['session_length']-1)) * (df['session_length']-df['action_num_reverse_chrono']-1)
#     df['log_recency_score'] = (2 ** linear_interpolation - 1).fillna(1.0)
#     return df

# def add_type_weighted_log_recency_score(df):
#     type_weights = {0:1, 1:6, 2:3}
#     df['type_weighted_log_recency_score'] = df['log_recency_score'] / df['type2id'].map(type_weights)
#     return df

# def apply(df, pipeline) :
#     for f in pipeline :
#         df = f(df)
    
#     return df


In [ ]:
# pipeline = [add_session_length, add_action_num_reverse_chrono, add_log_recency_score, add_type_weighted_log_recency_score]
# apply(train_sessions, pipeline)
# apply(test_sessions, pipeline)

In [ ]:
# train_sessions['gt']=0
# test_sessions['gt'] =1

In [ ]:
# train = train_sessions.merge(test_sessions, on=['session','ts','type', 'aid','type2id','session_length','action_num_reverse_chrono','log_recency_score','type_weighted_log_recency_score'], how='left')

# train['gt'] = train['gt'].fillna(0)
# train = train.sort_values('session').reset_index(drop=True)

In [ ]:
raw_corpus = []
for session, group_df in tqdm(train_sample.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))

for session, group_df in tqdm(test_sample.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))

In [ ]:
len(raw_corpus)

# 2.Word2Vec

documentation : https://radimrehurek.com/gensim/models/word2vec.html
reference : https://www.kaggle.com/code/takaito/otto-word2vec-tutorial

## create raw_corpus to train dataset of Word2vec

referece : https://www.kaggle.com/code/takaito/otto-word2vec-tutorial

In [ ]:
raw_corpus = []
for session, group_df in tqdm(train_sample.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))

for session, group_df in tqdm(test_sample.groupby(['session'])):
    raw_corpus.append(list(group_df['aid'].astype(str)))

In [ ]:
len(raw_corpus)

import library for Word2Vec

In [ ]:
import hashlib
import os
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
os.environ["PYTHONHASHSEED"] = str(42)
def hashfxn(x):
    return int(hashlib.md5(str(x).encode()).hexdigest(), 16)

## Train w2v model

vector_size : Dimensionality of the word vectors

window : Maxmum distance

min_count : Ignores all words with total frequency lower than this

sg : 1 for skip / otherwise cbow

i tried min_count 3, number 59625 not predicted in word2vec
when tried min_count 2, number 1024433 not predicted in word2vec

** with skip gram, it reduces error(prediction with Word2vec under 20) 5% to 2%.

In [ ]:
w2vec =Word2Vec(sentences=raw_corpus, 
                vector_size=100, window=5, 
                min_count=1, sg=1, workers=-1, 
                seed=42, hashfxn=hashfxn)

In [ ]:
w2vec.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")

Through AnnoyIndex function, we can measure similarity and find neighbors between train representation of our aid to create a submission.



Instance Attibute
AnnoyIndex(n_features, metric)

Instance Method
add_item(i,v)
- add item to be indexed
build(n_trees, n_jobs)

In [ ]:
%%time

from annoy import AnnoyIndex

aid2idx = {aid: i for i, aid in enumerate(w2vec.wv.index_to_key)}
index = AnnoyIndex(100, 'euclidean')

for aid, idx in aid2idx.items():
    index.add_item(idx, w2vec.wv.vectors[idx])
    
index.build(10,n_jobs=-1)

In [ ]:
# check how key and values matched in it.
aid2idx

In [ ]:
test_aids = test_sample.groupby('session')['aid'].apply(list)
test_types = test_sample.groupby('session')['type'].apply(list)

In [ ]:
test_aids

In [ ]:
test_types

this format referenced by @radek.
https://www.kaggle.com/code/radek1/word2vec-how-to-training-and-submission

defaultdict


In [ ]:
from collections import defaultdict

labels = []
type_weight_multipliers = {0: 1, 1: 6, 2: 3}

for AIDs, types in zip(test_aids,test_types):
#     print(AIDs,types)
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
        AIDs = list(dict.fromkeys(AIDs[::-1]))
        print(AIDs)
        # let's grab the most recent aid
        most_recent_aid = AIDs[0]
        print(most_recent_aid)
        # and look for some neighbors!
        nns = [w2vec.wv.index_to_key[i] for i in index.get_nns_by_item(aid2idx[most_recent_aid], 21)[1:]]
                        
        labels.append((AIDs+nns)[:20])

In [ ]:
# check the labelling is done well

cnt =0
for i in range(len(labels)) :
    if len(labels[i]) < 20 :
        cnt+=1

(cnt /len(labels) )*100

the error is 2.26%, with CBOW, error was 5.xx%

let's make submission files for word2vec

In [ ]:
session_types = ['clicks','carts','orders']
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_aids.index, 'labels': labels_as_strings, 's_type' : 0})

prediction_dfs = []

type2id = {'clicks' :0, "carts" : 1, 'orders' : 2}


for st in session_types:
    prediction = predictions.copy()
    prediction.session_type = prediction.session_type.astype('str') + f'_{st}'
    prediction.s_type = st
    prediction.s_type = prediction.s_type.apply(lambda x : type2id[x])
    prediction_dfs.append(prediction)
    
w2v_submission = pd.concat(prediction_dfs).reset_index(drop=True)
w2v_submission

In [ ]:
w2v_submission = w2v_submission.sort_values(['session_type', 's_type'], ascending=True)
w2v_submission

# 3. FM

references are here, but since i'm korean, there are some docs written in korean.

https://deepctr-doc.readthedocs.io/en/latest/Features.html#sparsefeat

https://deepctr-doc.readthedocs.io/en/latest/Examples.html

https://projectlog-eraser.tistory.com/41

https://arxiv.org/pdf/1703.04247v1.pdf

https://aplab.tistory.com/entry/%EC%B6%94%EC%B2%9C-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-%ED%83%90%EC%83%89-Deep-FM-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0

sparse : categorical value
dense : numerical value


## 3-1. vectorize train and test data

In [ ]:
# train_sessions.head()
# test_sessions['aid']=test_sessions['aid'].astype(int)

# train_sessions.to_parquet('train.parquet')
# test_sessions.to_parquet('test.parquet')

# train=pl.read_parquet('train.parquet')
# test=pl.read_parquet('test.parquet')

In [ ]:
%%time
train_pairs = (pl.concat([train, test])
    .groupby('session').agg([
        pl.col('aid'),
        pl.col('aid').shift(-1).alias('aid_next')
    ])
    .explode(['aid', 'aid_next'])
    .drop_nulls()
)[['aid', 'aid_next']]

In [ ]:
train_pairs.shape[0] /100000

cardinality that need to create the embedding layer

In [ ]:
cardinality_aids = max(train_pairs['aid'].max(),train_pairs['aid_next'].max())
cardinality_aids

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClickDataset(Dataset) :
    def __init__(self,pairs) :
        self.aid1 = pairs['aid'].to_numpy()
        self.aid2 = pairs['aid_next'].to_numpy()
    def __getitem__(self, idx) :
        aid1 = self.aid1[idx]
        aid2 = self.aid2[idx]
        return [aid1,aid2]
    def __len__(self) :
        return len(self.aid1)
    
train_ds = ClickDataset(train_pairs[:len(train_sessions)])
test_ds = ClickDataset(train_pairs[len(train_sessions):])

check how quickly we can iterate over a single epoch with a batch size 65535.

In [ ]:
train_ds = ClickDataset(train_pairs)
train_dl_pytorch = DataLoader(train_ds, 65535, True, num_workers=2)

In [ ]:
%%time

for batch in train_dl_pytorch :
    aid1,aid2 = batch[0], batch[1]

Load merlin

In [ ]:
!pip install merlin-dataloader==0.0.2

to using merlin-dataloader, create train_pairs and valid_pairs as parquet file.

In [ ]:
from merlin.loader.torch import Loader 
from merlin.io import Dataset

In [ ]:
train_pairs[:len(train_sessions)].to_pandas().to_parquet('train_pairs.parquet')
train_pairs[len(train_sessions):].to_pandas().to_parquet('valid_pairs.parquet')

In [ ]:
from merlin.io import Dataset

train_ds = Dataset('train_pairs.parquet')
train_dl_merlin = Loader(train_ds, 65536, True)

In [ ]:
%%time

for batch, _ in train_dl_merlin:
    aid1, aid2 = batch['aid'], batch['aid_next']

train matrix factorization model

documentation
- https://discuss.pytorch.org/t/indexing-embeddings-matrix-factorization/57268

In [ ]:
class MatrixFactorization(nn.Module) :
    def __init__(self, n_aids, n_factors) :
        super().__init__()
        self.aid_factors = nn.Embedding(n_aids, n_factors, sparse=True)
        
    def forward(self, aid1,aid2) :
        aid1 = self.aid_factors(aid1)
        aid2 = self.aid_factors(aid2)
        return (aid1*aid2).sum(dim=1)

class AverageMeter(object) :
    def __init__(self, name, fmt=':f') :
        self.name = name
        self.fmt =fmt
        self.reset()
        
    def reset(self) :
        self.avg=0
        self.val=0
        self.sum=0
        self.count = 0
    def update(self, val, n=1) :
        self.val = val
        self.sum +=val*n
        self.count +=n
        self.avg = self.sum / self.count
        
    def __str__(self) :
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
valid_ds = Dataset('valid_pairs.parquet')
valid_dl_merlin = Loader(valid_ds, 65536, True)

In [ ]:
from torch.optim import SparseAdam
import torch

num_epochs = 1
lr = 0.1

model = MatrixFactorization(cardinality_aids+1, 32)
optimizer = SparseAdam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
%%time

for epoch in range(num_epochs):
    for batch, _ in train_dl_merlin:
        model.train().cuda()
        losses = AverageMeter('Loss', ':.4e')
            
        aid1, aid2 = batch['aid'], batch['aid_next']
        output_pos = model(aid1, aid2)
        output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
        
        output = torch.cat([output_pos, output_neg])
        targets = torch.cat([torch.ones_like(output_pos), torch.zeros_like(output_pos)])
        loss = criterion(output, targets)
        losses.update(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    
    with torch.no_grad():
        accuracy = AverageMeter('accuracy')
        for batch, _ in valid_dl_merlin:
            aid1, aid2 = batch['aid'], batch['aid_next']
            output_pos = model(aid1, aid2)
            output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
            accuracy_batch = torch.cat([output_pos.sigmoid() > 0.5, output_neg.sigmoid() < 0.5]).float().mean()
            accuracy.update(accuracy_batch, aid1.shape[0])
            
    print(f'{epoch+1:02d}: * TrainLoss {losses.avg:.3f}  * Accuracy {accuracy.avg:.3f}')

embeddings

In [ ]:
embeddings = model.aid_factors.weight.detach().cpu().numpy()

check nearest neighbors search 

In [ ]:
%%time

from annoy import AnnoyIndex

fm = AnnoyIndex(32, 'euclidean')
for i, v in enumerate(embeddings) :
    fm.add_item(i,v)
    
fm.build(10)

In [ ]:
fm.get_nns_by_item(123,20)

create submission for Factorization Matrix

In [ ]:
from collections import defaultdict

session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type2id'].apply(list)

labels = []

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        # here we don't have 20 aids to output -- we will use approximate nearest neighbor search and our embeddings
        # to generate candidates!
        AIDs = list(dict.fromkeys(AIDs[::-1]))
        # let's grab the most recent aid
        most_recent_aid = AIDs[0]
        
        
        # and look for some neighbors!
        nns = fm.get_nns_by_item(most_recent_aid, 21)[1:]
                        
        labels.append((AIDs+nns)[:20])

In [ ]:
session_types = ['clicks','carts','orders']
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_aids.index, 'labels': labels_as_strings})

prediction_dfs = []


for st in session_types:
    prediction = predictions.copy()
    prediction.session_type = prediction.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(prediction)

fm_submission = pd.concat(prediction_dfs).reset_index(drop=True)

In [ ]:
fm_submission.head()

In [ ]:
w2v_submission.head()

sub for indexing

In [ ]:
sub = pd.merge(fm_submission, w2v_submission, on=['session_type', 'labels'], how='left')
sub = sub.set_index('session_type')

In [ ]:
w_labels = w2v_submission['labels']
w_labels = w_labels.str.split(' ')
f_labels= fm_submission['labels']
f_labels = f_labels.str.split(' ')

In [ ]:
sub_lists = []
for i in range(len(sub)) :
    sub_list = [item for sublist in zip(w_labels[i], f_labels[i]) for item in sublist]
    sub_list = list(dict.fromkeys(sub_list))
    sub_lists.append(sub_list[:20])
sub_lists[:10]

In [ ]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in sub_lists]
submission = pd.DataFrame(data={'session_type': sub.index, 'labels': labels_as_strings})

submission = submission.sort_values('session_type', axis=0, ascending=True)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission